In [1]:
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from IPython.display import Image, display, clear_output
#%matplotlib notebook

In [2]:
h5Tmp = 'tmp.h5'

def tracerGraphes(listeEssais, typeCourbe='ProcessedCurve',
                  indiceX='1. Dépl. (mm)', 
                  indiceY='2. Temps (s)'):
    f = h5py.File(h5Tmp, 'r')
    typeEssai = typeEssais.value   
    try:
        if (len(listeEssais) > 0) and (not 'None' in listeEssais):
            fig, ax = plt.subplots(figsize=(14,8))
            p, leg = list(), list()            
            nomPara, uniPara = list(), list()
            for essai in listeEssais:
                curveProc = f[typeEssai + '/' + essai + '/' + typeCourbe]
                for i, para in enumerate(curveProc.attrs.keys()):
                    name = curveProc.attrs.get(para)
                    if i%2 == 0:
                        nom = name.decode("ISO-8859-1")
#                         if nom == '%':
#                             nom = '\%'
                        nomPara.append(nom)
                    else:
                        uni = name.decode("ISO-8859-1")
#                         if uni == '%':
#                             uni = '\%'
                        uniPara.append(uni)

                data = curveProc['DATA']
                pns, = ax.plot(data[:, eval(indiceX[0]) - 1], 
                               data[:, eval(indiceY[0]) - 1],
                               marker='None',
                               linestyle='-',
                               linewidth=4, mew=2,
                               markerfacecolor='None',
                               markersize=10)
                
                p.append(pns)
                leg.append(essai.replace('_', ' '))

            ax.legend(p, leg, loc='best', prop={"size": 15},
                      ncol=1, numpoints=1)
            plt.grid(True, which="both", linestyle='--')

            ax.set_xlabel(nomPara[eval(indiceX[0]) - 1] +
                          ' (' + uniPara[eval(indiceX[0]) - 1] + ')',
                          fontsize=20)
            ax.set_ylabel(nomPara[eval(indiceY[0]) - 1] +
                          ' (' + uniPara[eval(indiceY[0]) - 1] + ')',
                          fontsize=20)
            
            # Tailles des ticks en format scientifique
            ax.tick_params(labelsize=20)
            ax.set_xlim(0,)
            ax.set_ylim(0,)

            tx, ty = ax.xaxis.get_offset_text(), ax.yaxis.get_offset_text()
            tx.set_size(20)
            ty.set_size(20)

            # Epaisseur des axes
            for axis in ['top', 'bottom', 'left', 'right']:
                plt.gca().spines[axis].set_linewidth(2)
            plt.subplots_adjust(bottom=0.15)
            # plt.subplots_adjust(right=0.8)
            plt.subplots_adjust(left=0.15)
        
            show_inline_matplotlib_plots()
            
    except KeyError:
        print('')

In [3]:
fichierVers = widgets.FileUpload(
    accept='.h5',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False, # True to accept multiple files upload else False
    layout=widgets.Layout(width='80%', height='40px'),
    description='Choisir un fichier HDF5',
    style = {'description_width': 'initial'}
)
nomFichierVers = widgets.Text(disabled=True,
                              description='Fichier téléversé',
                              layout=widgets.Layout(width='80%', height='40px'),
                              style = {'description_width': 'initial'})

In [4]:
typeEssais = widgets.Dropdown(
    options=['--'],
    value='--',
    #rows=10,
    description="Type d'essais",
    disabled=False,
    layout=widgets.Layout(width='80%', height='40px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

chercherEssai = widgets.Text(placeholder='Taper un mot',
                             description= 'Chercher les essais:',
                             layout=widgets.Layout(width='80%', height='40px',
                                                   visibility='hidden'),
                             style = {'description_width': 'initial'})

chercherAtt = widgets.Text(placeholder='Taper un mot',
                           description= "Chercher l'attribute:",
                           layout=widgets.Layout(width='80%', height='40px',
                                                 visibility='hidden'),
                           style = {'description_width': 'initial'})

essaisTS = widgets.SelectMultiple(
    options=['--'],
    value=['--'],
    #rows=10,
    description='Essais',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

essaisTSSolo = widgets.Select(
    options=['--'],
    value='--',
    #rows=10,
    description='Essais',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

attributes = widgets.Select(
    description='Attributes',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

attributeVal = widgets.Text(
    description="Valeur de l'attribute",
    disabled=True,
    layout=widgets.Layout(width='80%', height='40px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

typeCourbe = widgets.Dropdown(
    options=['--'],
    value='--',
    #rows=10,
    description='Type de courbe',
    disabled=False,
    layout=widgets.Layout(width='80%', height='40px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

abscisse = widgets.Dropdown(
    options=['--'],
    value='--',
    #rows=10,
    description='Abscisse',
    disabled=False,
    layout=widgets.Layout(width='80%', height='40px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

ordonnee = widgets.Dropdown(
    options=['--'],
    value='--',
    #rows=10,
    description='Ordonnée',
    disabled=False,
    layout=widgets.Layout(width='80%', height='40px', visibility='hidden'),
    style = {'description_width': 'initial'}
)

graphes = widgets.interactive_output(tracerGraphes, {'listeEssais': essaisTS, 
                                                     'typeCourbe': typeCourbe,
                                                     'indiceX': abscisse,
                                                     'indiceY': ordonnee}) 
graphes.layout.visibility = 'hidden'

def changeFichier(b=None):
    [dataEssais] = fichierVers.metadata
    nomFichier = dataEssais['name']
    nomFichierVers.layout.visibility = 'visible'
    nomFichierVers.value = nomFichier
    with open(h5Tmp, 'wb') as f:
        valEssai = fichierVers.value
        contenu = valEssai[nomFichier]['content']
        f.write(contenu)
        f.close()
    choisirTypeEssai(h5Tmp)
    typeEssais.layout.visibility = 'visible'
    
def choisirTypeEssai(nomFichier):
    f = h5py.File(nomFichier, 'r')
    typeE = [tE for tE in f.keys()]
    typeEssais.options = typeE
    typeEssais.value = typeE[0]

def choisirEssai(typeEssai):
    try:
        f = h5py.File(h5Tmp, 'r')
        essaisTS.options = [e for e in f[typeEssai].keys()]
        essaisTSSolo.options = [e for e in f[typeEssai].keys()]
        
        if typeEssai == 'None' or len(essaisTS.options)==0:
            essaisTS.layout.visibility = 'hidden'
            essaisTSSolo.layout.visibility = 'hidden'
            typeCourbe.layout.visibility = 'hidden'
            abscisse.layout.visibility = 'hidden'
            ordonnee.layout.visibility = 'hidden'
            attributes.layout.visibility = 'hidden'
            attributeVal.layout.visibility = 'hidden'
            chercherAttr.layout.visibility = 'hidden'
            chercherEssai.layout.visibility = 'hidden'
        else:
            typeCourbe.options =\
                [c for c in f[typeEssai + '/' + essaisTS.options[0]].keys()]
            typeCourbe.layout.visibility = 'visible'
            essaisTS.layout.visibility = 'visible'
            essaisTSSolo.layout.visibility = 'visible'            
            abscisse.layout.visibility = 'visible'
            ordonnee.layout.visibility = 'visible'
            attributes.layout.visibility = 'visible'
            attributeVal.layout.visibility = 'visible'
            chercherAtt.layout.visibility = 'visible'
            chercherEssai.layout.visibility = 'visible'

    except ValueError:
        print('Choisir un fichier HDF5')

def choisirXY(typeEssai, typeCourbe):
    print(typeEssai, typeCourbe)
    try:
        f = h5py.File(h5Tmp, 'r')
        essais = [e for e in f[typeEssai].keys()]
        
        if len(essais)!=0:
            curveProc = f[typeEssai + '/' + essais[0] + '/' + typeCourbe]
            nomPara, uniPara = list(), list()
            for i, para in enumerate(curveProc.attrs.keys()):
                name = curveProc.attrs.get(para)
                if i%2 == 0:
                    nom = name.decode("ISO-8859-1")
                    nomPara.append(nom)
                else:
                    uni = name.decode("ISO-8859-1")
                    uniPara.append(uni)
            print(nomPara)           
            abscisse.options = [str(j + 1) + '. ' + 
                                nomPara[j] + ' (' + 
                                uniPara[j] + ')' for j in range(len(nomPara))]
            ordonnee.options = [str(j + 1) + '. ' + 
                                nomPara[j] + ' (' + 
                                uniPara[j] + ')' for j in range(len(nomPara))]

    except ValueError:
        print('Choisir un fichier HDF5')

def listeAttr(typeEssai, nomEssai):
    try:
        f = h5py.File(h5Tmp, 'r')
        essaiDat = f[typeEssai + '/' + nomEssai]
        lAtt = [a for a in essaiDat.attrs.keys()]
        attVal = list()
        for a in essaiDat.attrs.keys():
            try:
                attVal.append(essaiDat.attrs.get(a).decode("ISO-8859-1"))
            except AttributeError:
                attVal.append(str(essaiDat.attrs.get(a)))
                
        if typeEssai == 'None' or len(lAtt)==0:
            attributes.layout.visibility = 'hidden'
            attributeVal.layout.visibility = 'hidden'
        else:
            attributes.layout.visibility = 'visible'
            attributeVal.layout.visibility = 'visible'
            attributes.options = lAtt
    except ValueError:
        print('Choisir un fichier HDF5')

def getAttrVal(typeEssai, nomEssai, attribute):
    try:
        f = h5py.File(h5Tmp, 'r')
        essaiDat = f[typeEssai + '/' + nomEssai]
        lAtt = [a for a in essaiDat.attrs.keys()]
        if typeEssai == 'None' or len(lAtt)==0:
            attributeVal.layout.visibility = 'hidden'
        else:
            attributeVal.layout.visibility = 'visible'
            try:
                attributeVal.value = essaiDat.attrs.get(attribute).decode("ISO-8859-1")
            except AttributeError:
                attributeVal.value = str(essaiDat.attrs.get(attribute))
    except ValueError:
        print('Choisir un fichier HDF5')

In [5]:
def cacherGraphe(typeEssai, essaisTS):
    try:
        f = h5py.File(h5Tmp, 'r')
        if len(f[typeEssai].keys()) and len(essaisTS) > 0:
            graphes.layout.visibility = 'visible'        
    except ValueError:
        print('Choisir un fichier HDF5')

bouton = widgets.Button(description="Téléverser le fichier HDF5",
                        button_style = 'success',
                        layout=widgets.Layout(width='80%', height='40px'),
                        style = {'description_width': 'initial'})
# Essai individuel
@bouton.on_click
def graphe_on_click(b):
    """
    Bouton pour la creation du graphe
    """
    with out:
        clear_output()
        try:
            changeFichier()
        except ValueError:
            print('Choisir un fichier HDF5 !')
    
widgets.interactive(choisirEssai, typeEssai=typeEssais)
widgets.interactive(choisirXY, typeEssai=typeEssais, typeCourbe=typeCourbe)
widgets.interactive(cacherGraphe, typeEssai=typeEssais, essaisTS=essaisTS)
widgets.interactive(listeAttr, typeEssai=typeEssais, nomEssai=essaisTSSolo)
widgets.interactive(getAttrVal, typeEssai=typeEssais, nomEssai=essaisTSSolo, attribute=attributes)

def texteChange(change):
    """Dynamically update the widget experiments - Essais"""
    search_input = change['new']
    typeEssai = typeEssais.value
    f = h5py.File(h5Tmp, 'r')
    essaisListe = [e for e in f[typeEssai].keys()]
    
    if search_input == '':
        close_matches = essaisListe
        # Reset search field, default to last 9 experiments
    else:
        # Filter by search
        close_matches = [x for x in essaisListe if search_input.lower() in x.lower()]

    essaisTS.options = close_matches
    essaisTSSolo.options = close_matches

def texteChangeAtt(change):
    """Dynamically update the widget experiments - Attributes"""
    search_input = change['new']
    typeEssai = typeEssais.value
    nomEssai = essaisTSSolo.value
    
    f = h5py.File(h5Tmp, 'r')
    essaiDat = f[typeEssai + '/' + nomEssai]
    lAtt = [a for a in essaiDat.attrs.keys()]
      
    if search_input == '':
        close_matches = lAtt
        # Reset search field, default to last 9 experiments
    else:
        # Filter by search
        close_matches = [x for x in lAtt if search_input.lower() in x.lower()]

    attributes.options = close_matches
    
chercherEssai.observe(texteChange, names='value')
chercherAtt.observe(texteChangeAtt, names='value')

out = widgets.Output()

In [6]:
grid1 = widgets.GridspecLayout(2, 2)
grid2 = widgets.GridspecLayout(3, 2)
grid3 = widgets.GridspecLayout(3, 2)

In [ ]:
# fill it in with widgets
grid1[0, 0] = fichierVers
grid1[1, 0] = bouton
grid1[0, 1] = nomFichierVers
grid1[1, 1] = typeEssais

grid2[0, 0] = chercherEssai
grid2[1:, 0] = essaisTS
grid2[0, 1] = typeCourbe
grid2[1, 1] = abscisse
grid2[2, 1] = ordonnee

grid3[0, 0] = chercherEssai
grid3[1:, 0] = essaisTSSolo
grid3[0, 1] = chercherAtt
grid3[1, 1] = attributes
grid3[2, 1] = attributeVal

tab1 = widgets.VBox(children=[grid2, graphes])

tabs = widgets.Tab(children=[tab1, grid3])
tabs.set_title(0, 'Courbes')
tabs.set_title(1, 'Attributes')

In [7]:
display(grid1, tabs)

GridspecLayout(children=(FileUpload(value={}, accept='.h5', description='Choisir un fichier HDF5', layout=Layo…